In [9]:
import pandas as pd
# df=pd.read_excel("data-preterm.xls")
df=pd.read_csv("data-preterm.csv")

In [10]:
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc,roc_auc_score
import xgboost as xgb
from xgboost import plot_importance
from sklearn.ensemble import RandomForestClassifier

In [4]:
!pip install xlrd

  Obtaining dependency information for xlrd from https://files.pythonhosted.org/packages/a6/0c/c2a72d51fe56e08a08acc85d13013558a2d793028ae7385448a6ccdfae64/xlrd-2.0.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---------------- ----------------------- 41.0/96.5 kB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 96.5/96.5 kB 1.4 MB/s eta 0:00:00


In [71]:
df.head()

,GEST,DILATE,EFFACE,CONSIS,CONTR,MEMBRAN,INFECTION,CHR_HYPERTENSION,AGE,STRAT,GRAVID,PARIT,DIAB,TRANSF,GEMEL,PREMATURE
0,31.0,3.0,100.0,3.0,1.0,2.0,1.0,1.0,26.0,3.0,1.0,0.0,2.0,2.0,1.0,positif
1,28.0,8.0,0.0,3.0,1.0,2.0,1.0,1.0,25.0,3.0,1.0,0.0,2.0,1.0,2.0,positif
2,31.0,3.0,100.0,3.0,2.0,2.0,1.0,1.0,28.0,3.0,2.0,0.0,2.0,1.0,1.0,positif
3,27.0,2.0,75.0,3.0,2.0,2.0,1.0,1.0,27.0,2.0,2.0,1.0,2.0,1.0,2.0,positif
4,28.0,6.0,75.0,3.0,2.0,2.0,1.0,1.0,17.0,3.0,1.0,0.0,2.0,1.0,1.0,positif


In [72]:
df.columns.tolist()

['GEST',
 'DILATE',
 'EFFACE',
 'CONSIS',
 'CONTR',
 'MEMBRAN',
 'INFECTION',
 'CHR_HYPERTENSION',
 'AGE',
 'STRAT',
 'GRAVID',
 'PARIT',
 'DIAB',
 'TRANSF',
 'GEMEL',
 'PREMATURE']

In [11]:
# data2016 = data2016.replace({'N': 0, 'Y': 1, 'U': np.nan, 'X':np.nan})
df=df.replace({'positif':1,'negatif':0})
df.head()

,GEST,DILATE,EFFACE,CONSIS,CONTR,MEMBRAN,INFECTION,CHR_HYPERTENSION,AGE,STRAT,GRAVID,PARIT,DIAB,TRANSF,GEMEL,PREMATURE
0,31.0,3.0,100.0,3.0,1.0,2.0,1.0,1.0,26.0,3.0,1.0,0.0,2.0,2.0,1.0,1
1,28.0,8.0,0.0,3.0,1.0,2.0,1.0,1.0,25.0,3.0,1.0,0.0,2.0,1.0,2.0,1
2,31.0,3.0,100.0,3.0,2.0,2.0,1.0,1.0,28.0,3.0,2.0,0.0,2.0,1.0,1.0,1
3,27.0,2.0,75.0,3.0,2.0,2.0,1.0,1.0,27.0,2.0,2.0,1.0,2.0,1.0,2.0,1
4,28.0,6.0,75.0,3.0,2.0,2.0,1.0,1.0,17.0,3.0,1.0,0.0,2.0,1.0,1.0,1


In [86]:
# df.isna().sum()
df['GEST'].max()

35.0

In [5]:
df[df['PREMATURE'] == 0]

,GEST,DILATE,EFFACE,CONSIS,CONTR,MEMBRAN,INFECTION,CHR_HYPERTENSION,AGE,STRAT,GRAVID,PARIT,DIAB,TRANSF,GEMEL,PREMATURE
8,33.0,0.0,25.0,3.0,1.0,2.0,0.0,0.0,22.0,4.0,1.0,0.0,2.0,2.0,2.0,0
14,30.0,0.0,25.0,3.0,1.0,2.0,0.0,0.0,32.0,3.0,3.0,1.0,2.0,2.0,1.0,0
18,30.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,32.0,3.0,2.0,0.0,2.0,2.0,1.0,0
20,32.0,0.0,0.0,3.0,1.0,2.0,0.0,0.0,24.0,4.0,3.0,1.0,2.0,2.0,1.0,0
21,33.0,3.0,50.0,3.0,1.0,2.0,0.0,0.0,17.0,4.0,1.0,0.0,2.0,2.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,34.0,2.0,50.0,3.0,1.0,2.0,0.0,0.0,20.0,4.0,2.0,1.0,2.0,1.0,2.0,0
373,32.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,21.0,4.0,3.0,1.0,2.0,1.0,1.0,0
375,32.0,1.0,75.0,2.0,1.0,2.0,0.0,0.0,31.0,4.0,6.0,2.0,2.0,2.0,1.0,0
380,34.0,0.0,50.0,2.0,1.0,2.0,0.0,0.0,32.0,4.0,1.0,0.0,2.0,2.0,1.0,0


In [39]:
df.corr()['PREMATURE'].sort_values()

MEMBRAN            -0.246504
TRANSF             -0.239941
PARIT              -0.171479
STRAT              -0.101656
AGE                -0.094968
GRAVID             -0.085460
GEST               -0.077971
DIAB               -0.015762
CONTR               0.066848
GEMEL               0.154172
CONSIS              0.203372
DILATE              0.298831
EFFACE              0.337076
CHR_HYPERTENSION    0.371554
INFECTION           0.768031
PREMATURE           1.000000
Name: PREMATURE, dtype: float64

In [12]:
train,test = split(df,train_size = 0.7)
X_train = train.drop(columns = ['PREMATURE'])
y_train = train.PREMATURE
X_test = test.drop(columns = ['PREMATURE'])
y_test = test.PREMATURE

In [138]:
X_train.shape

(273, 15)

In [13]:
scaler = MaxAbsScaler()
sclr = scaler.fit(X_train)
X_train = sclr.transform(X_train)
X_test = sclr.transform(X_test)

In [42]:
rfc = RandomForestClassifier(max_depth=8,n_estimators=10).fit(X_train,y_train)
rfc_auc = roc_auc_score(y_test, rfc.predict(X_test))
rfc_auc
# ypred=rfc.predict(X_test)
# print(ypred[10:20])
# print(y_test[10:20])

0.8212643678160918

In [43]:
xgbm = xgb.XGBModel().fit(X_train,y_train)
xgb_auc = roc_auc_score(y_test, xgbm.predict(X_test).round())
xgb_auc

0.8270114942528736

In [44]:
from sklearn.naive_bayes import GaussianNB

# Build a Gaussian Classifier
nb = GaussianNB()

# Model training
nb.fit(X_train, y_train)

nb_auc = roc_auc_score(y_test, nb.predict(X_test))
nb_auc

0.7879310344827586

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=4).fit(X_train, y_train)
knn_auc=roc_auc_score(y_test, knn.predict(X_test))
knn_auc

0.8155172413793103

In [14]:
import tensorflow as tf

array([[0.71428571, 0.        , 0.5       , ..., 0.22222222, 0.5       ,
        0.5       ],
       [0.74285714, 0.25      , 0.3       , ..., 0.22222222, 1.        ,
        0.5       ],
       [0.8       , 0.        , 0.5       , ..., 0.22222222, 0.5       ,
        1.        ],
       ...,
       [0.82857143, 0.        , 0.        , ..., 0.22222222, 1.        ,
        1.        ],
       [0.71428571, 0.125     , 0.1       , ..., 0.22222222, 1.        ,
        0.5       ],
       [0.82857143, 0.125     , 0.25      , ..., 0.22222222, 1.        ,
        0.5       ]])

In [15]:
# from tf.keras.models import Sequential
from tensorflow import keras
# from tensorflow.keras.optimizers import Adam
# from  keras import models
from keras.models import Sequential
from keras.layers import Dense
# from keras.optimizers import adam
# from keras.optimizers import adam_v2
# from keras import optimizers
# from keras.optimizers import RMSprop
from keras.regularizers import l2
# opt = RMSprop(learning_rate=0.0001, decay=1e-6)
# optimizer = optimizers.Adam()
model = Sequential()
model.add(Dense(8, activation='relu',kernel_regularizer=l2(0.01), input_shape=(15,)))
model.add(Dense(8, activation='relu',kernel_regularizer=l2(0.01),))
model.add(Dense(1, activation='sigmoid'))
# opt = optimizers.adam_v2.Adam(lr=0.001, decay=1e-6)
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])
model.fit(X_train, y_train,epochs=4, batch_size=1, verbose=1)

Epoch 1/4


C:\Users\gopal\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


273/273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6086 - loss: 0.7997
Epoch 2/4
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7621 - loss: 0.5351
Epoch 3/4
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8908 - loss: 0.4174
Epoch 4/4
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8990 - loss: 0.3718


In [125]:
# import pickle

# # save the iris classification model as a pickle file
# model_pkl_file = "preterm_model.pkl"  

# with open(model_pkl_file, 'wb') as file:  
#     pickle.dump(model, file)
model.save('my_model.keras')

In [126]:
model.save('my_model.h5')

In [77]:
score = model.evaluate(X_test, y_test,verbose=1)
print(score)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9082 - loss: 0.2877 
[0.28958961367607117, 0.9059829115867615]


In [84]:
# y_pred = model.predict(X_test)
# y_pred
predictions = (model.predict(X_test) > 0.5).astype("int32")
predictions

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
    

In [16]:
 data = {
        "GEST": 28,
        "DILATE": 0,
        "EFFACE": 0,
        "CONSIS": 2,
        "CONTR":1,
        "MEMBRAN":2,
        "INFECTION":0,
        "CHR_HYPERTENSION":0,
        "AGE":23,
        "STRAT":2,
        "GRAVID":0,
        "PARIT":0,
        "DIAB":2,
        # "BEBAGE":BEBAGE,
        "TRANSF":2,
        "GEMEL":1


    }
features = pd.DataFrame(data, index=[0])
features

,GEST,DILATE,EFFACE,CONSIS,CONTR,MEMBRAN,INFECTION,CHR_HYPERTENSION,AGE,STRAT,GRAVID,PARIT,DIAB,TRANSF,GEMEL
0,28,0,0,2,1,2,0,0,23,2,0,0,2,2,1


In [17]:
predictions = (model.predict(features) > 0.5).astype("int32")
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


array([[0]])

In [18]:
model.save('my_model.h5')